In [ ]:
import json
import os
import datetime

# Custom exception class for handling invalid data.
class InvalidDataException(Exception):
    def __init__(self, message="Error loading/saving data. Please check the data file."):
        self.message = message
        super().__init__(self.message)

# Custom exception class for handling invalid date formats.
class InvalidDateException(Exception):
    def __init__(self, message="Invalid date format. Date must be in the format MM/DD/YYYY."):
        self.message = message
        super().__init__(self.message)
        
# Custom exception class for handling cases where a plant is not found.
class PlantNotFoundException(Exception):
    def __init__(self, message="Plant not found."):
        self.message = message
        super().__init__(self.message)


def load_data():
    """
    Load data from the 'garden_data.json' file if it exists,
    or return a default data structure if the file doesn't exist.
    """
    try:
        if os.path.exists("garden_data.json"):
            with open("garden_data.json", "r") as file:
                return json.load(file)
        else:
            return {"plants": []}
    except (FileNotFoundError, json.decoder.JSONDecodeError):
        raise InvalidDataException("Error loading data. Please check the data file.")

def save_data(data):
    """
    Save the data to the 'garden_data.json' file.
    """
    try:
        with open("garden_data.json", "w") as file:
            json.dump(data, file)
    except (FileNotFoundError, json.decoder.JSONDecodeError):
        raise InvalidDataException("Error saving data. Please check the data file.")

def is_valid_date(date_string):
    """
    Check if a date string is in the format 'MM/DD/YYYY'.
    """
    try:
        datetime.datetime.strptime(date_string, "%m/%d/%Y")
        return True
    except ValueError:
        return False

def add_plant(data):
    """
    Add a new plant to the garden.
    """
    try:
        name = input("Enter the plant name: ")

        if data["plants"]:
            # Check if the plant already exists
            for plant in data["plants"]:
                if plant is not None and plant.get("name", "").lower() == name.lower():
                    print(f"The plant '{name}' already exists in the garden.")
                    return

        quantity = int(input("Enter the quantity of this plant: "))
        watered = input("Enter the last watering date (MM/DD/YYYY): ")
        fed = input("Enter the last feeding date (MM/DD/YYYY): ")

        if not is_valid_date(watered) or not is_valid_date(fed):
            raise InvalidDateException("Date format must be MM/DD/YYYY.")    # Check if the entered watering and feeding dates have a valid format.

        activity_log = []

        new_plant = {"name": name, "quantity": quantity, "last_watered": watered, "last_fed": fed, "activity_log": activity_log}

        data["plants"].append(new_plant)
        save_data(data)

        print("Plant added successfully!")

    except ValueError:
        raise InvalidDataException("Quantity must be a valid integer.")
    except InvalidDateException as e:
        raise InvalidDataException(str(e))


def log_activity(plant):
    """
    Log an activity for a plant.
    """
    try:
        activity = input("Enter the activity (e.g., watered, fed): ")
        date = input("Enter the date of the activity (MM/DD/YYYY): ")

        if not is_valid_date(date):
            raise InvalidDateException("Date format must be MM/DD/YYYY.")

        plant["activity_log"].append({"activity": activity, "date": date})

    except InvalidDateException as e:
        raise InvalidDataException(str(e))

def view_activity_log(plant):
    """
    View activity log for a plant.
    """
    if plant["activity_log"]:
        print(f"\nActivity Log for {plant['name']} ({plant['quantity']} plants):")
        for activity in plant["activity_log"]:
            print(f"{activity['activity']} on {activity['date']}")
    else:
        print(f"No activity log for {plant['name']} yet.")

def delete_plant(data):
    """
    Delete a plant from the garden.
    """
    try:
        if data["plants"]:
            plant_index = int(input("Enter the index of the plant to delete: ")) - 1
            if 0 <= plant_index < len(data["plants"]):
                del data["plants"][plant_index]
                save_data(data)
                print("Plant deleted successfully!")
            else:
                raise PlantNotFoundException("Invalid plant index.")
        else:
            raise PlantNotFoundException("No plants to delete.")
    except (ValueError, PlantNotFoundException) as e:
        print(f"Error: {e}")

def edit_plant(data):
    """
    Edit details of a specific plant.
    """
    try:
        if data["plants"]:
            plant_index = int(input("Enter the index of the plant to edit: ")) - 1
            if 0 <= plant_index < len(data["plants"]):
                plant = data["plants"][plant_index]
                print(f"\nEditing details for {plant['name']}:")

                plant["quantity"] = int(input(f"Enter new quantity ({plant['quantity']}): "))
                plant["last_watered"] = input(f"Enter new last watering date ({plant['last_watered']}): ")
                plant["last_fed"] = input(f"Enter new last feeding date ({plant['last_fed']}): ")

                save_data(data)
                print("Plant details updated successfully!")
            else:
                raise PlantNotFoundException("Invalid plant index.")
        else:
            raise PlantNotFoundException("No plants to edit.")
    except (ValueError, PlantNotFoundException) as e:
        print(f"Error: {e}")


def search_plant(data):
    """
    Search for plants by name.
    """

def main():
    try:
        data = load_data()

        while True:
            # Display menu options
            print("\nKitchen Garden Manager")
            # (other menu options...)
            print("1. Add a Plant")
            print("2. View Plants")
            print("3. Log Activity")
            print("4. View Activity Log")
            print("5. Delete a plant")
            print("6. Edit a plant")
            print("7. Search a plant")
            print("8. Quit")

            choice = input("Enter your choice (1/2/3/4/5/6/7/8): ")   # Prompts user for choice

            if choice == "1":
                try:
                    plant_data = add_plant(data)
                    data["plants"].append(plant_data)
                    save_data(data)
                    print("Plant added successfully!")
                except InvalidDataException as e:
                    print(f"Error: {e}")

            elif choice == "2":
                # Display information for each plant
                if data["plants"]:
                    print("\nPlants in your garden:")
                    for idx, plant in enumerate(data["plants"], start=1):
                        print(f"\nPlant {idx}:")
                        if plant is not None:
                            print(f"Name: {plant['name']}")
                            print(f"Quantity: {plant['quantity']}")
                            print(f"Last Watered: {plant['last_watered']}")
                            print(f"Last Fed: {plant['last_fed']}")
                        else:
                            print("Plant data is incomplete.")
                else:
                    print("No plants in your garden yet.")

            elif choice == "3":  # log activity details for plants. Catch if invalid index is given
                try:
                    if data["plants"]:
                        plant_index = int(input("Enter the index of the plant to log activity: ")) - 1
                        if 0 <= plant_index < len(data["plants"]):
                            try:
                                log_activity(data["plants"][plant_index])
                                save_data(data)
                                print("Activity logged successfully!")
                            except InvalidDataException as e:
                                print(f"Error: {e}")
                        else:
                            raise PlantNotFoundException("Invalid plant index.")
                    else:
                        raise PlantNotFoundException("No plants to log activity for.")
                except (ValueError, PlantNotFoundException) as e:
                    print(f"Error: {e}")

            elif choice == "4":
                try:
                    if data["plants"]:
                        plant_index = int(input("Enter the index of the plant to view activity log: ")) - 1
                        if 0 <= plant_index < len(data["plants"]):
                            view_activity_log(data["plants"][plant_index])
                        else:
                            raise PlantNotFoundException("Invalid plant index.")
                    else:
                        raise PlantNotFoundException("No plants to view activity log for.")
                except (ValueError, PlantNotFoundException) as e:
                    print(f"Error: {e}")

            elif choice == "5":  # Delete plant
                try:
                    delete_plant(data)
                except InvalidDataException as e:
                    print(f"Error: {e}")

            elif choice == "6":  # Edit plant
                try:
                    edit_plant(data)
                except InvalidDataException as e:
                    print(f"Error: {e}")

            elif choice == "7":  # Search plant
                try:
                    search_plant(data)
                except InvalidDataException as e:
                    print(f"Error: {e}")

            elif choice == "8":
                break

            else:
                print("Invalid choice. Please try again.")

    except InvalidDataException as e:
        print(f"Error: {e}")


if __name__ == "__main__":
    main()



Kitchen Garden Manager
1. Add a Plant
2. View Plants
3. Log Activity
4. View Activity Log
5. Delete a plant
6. Edit a plant
7. Search a plant
8. Quit
Enter your choice (1/2/3/4/5/6/7/8): 7

Kitchen Garden Manager
1. Add a Plant
2. View Plants
3. Log Activity
4. View Activity Log
5. Delete a plant
6. Edit a plant
7. Search a plant
8. Quit
Enter your choice (1/2/3/4/5/6/7/8): 7

Kitchen Garden Manager
1. Add a Plant
2. View Plants
3. Log Activity
4. View Activity Log
5. Delete a plant
6. Edit a plant
7. Search a plant
8. Quit
Enter your choice (1/2/3/4/5/6/7/8): 6
Enter the index of the plant to edit: 8
Error: Invalid plant index.

Kitchen Garden Manager
1. Add a Plant
2. View Plants
3. Log Activity
4. View Activity Log
5. Delete a plant
6. Edit a plant
7. Search a plant
8. Quit
Enter your choice (1/2/3/4/5/6/7/8): 2

Plants in your garden:

Plant 1:
Name: rose
Quantity: 3
Last Watered: 10/2/2023
Last Fed: 10/2/2023

Plant 2:
Name: Hibiscus
Quantity: 18
Last Watered: 10/2/2023
Last Fed: